This notebook processes the output by Picturae.

In [1]:
import pandas as pd
import glob
import json
import os
import shutil

In [4]:
def process_description(x):
    return pd.json_normalize(json.loads(x)).iloc[:, 1:2].values[0][0]


def fix_labels(x):
    return x.lower().replace(' ','_')

In [6]:
input_path = '../data/intermediary/annotations_step2/'
predicted_labels = pd.read_json('../data/processed/20220301predictions.json', lines=True)

In [7]:
top_1 = []
top_5 = []
for k, v in predicted_labels['predictions'].to_dict().items():
    top_1.append(list(v.keys())[-1])
    top_5.append(list(v.keys()))


predicted_labels['top_1'] = top_1
predicted_labels['top_5'] = top_5

## Images
id = image_id

In [8]:
images = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_images.csv'))

## Indexeer data

In [9]:
indexeer_data = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_indexeer_data.csv'))

In [11]:
indexeer_data['annotated_label'] = indexeer_data['descriptions'].apply(process_description)

## Probleem gevallen

In [12]:
probleem_gevallen = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_probleemgevallen.csv'))

probleem gevallen lijkt niet relevant

## Scans

In [14]:
scans = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_scans.csv'))

## Scans Images

In [15]:
scans_images = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_scans_images.csv'))

## opmerkingen

In [16]:
opmerkingen = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_opmerkingen.csv'))

## Controle data

In [17]:
controle_data = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_controle_data.csv'))

In [18]:
controle_data['corrected_label'] = controle_data['descriptions'].apply(process_description)

## Scan acties

In [19]:
scan_acties = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_scan_acties.csv'))

## Series

In [20]:
series = pd.read_csv(os.path.join(input_path, 'pic_vh_nl_prod_ranh_tagcorrection_deboer_table_series.csv'))

In [21]:
series[series['titel'].str.contains('batch2')].head(10)

,id,pad,naam,titel,formulier_id,status,completed_on,created_at,updated_at
20,21,NaN,random_batch2_1,random_batch2_1,1,103,2022-05-09 09:33:41,2022-03-28 12:30:50,2022-05-09 09:33:41
21,22,NaN,random_batch2_10,random_batch2_10,1,103,2022-05-09 09:33:41,2022-03-28 12:32:03,2022-05-09 09:33:41
22,23,NaN,random_batch2_11,random_batch2_11,1,103,2022-05-09 09:33:41,2022-03-28 12:33:27,2022-05-09 09:33:41
23,24,NaN,random_batch2_12,random_batch2_12,1,103,2022-05-16 11:53:15,2022-03-28 12:34:54,2022-05-16 11:53:15
24,25,NaN,random_batch2_13,random_batch2_13,1,103,2022-05-16 11:53:15,2022-03-28 12:36:12,2022-05-16 11:53:15
25,26,NaN,random_batch2_14,random_batch2_14,1,103,2022-05-09 09:33:41,2022-03-28 12:37:19,2022-05-09 09:33:41
26,27,NaN,random_batch2_15,random_batch2_15,1,103,2022-05-09 09:33:41,2022-03-28 12:38:26,2022-05-09 09:33:41
27,28,NaN,random_batch2_16,random_batch2_16,1,103,2022-05-16 11:53:15,2022-03-28 12:39:34,2022-05-16 11:53:15
28,29,NaN,random_batch2_17,random_batch2_17,1,103,2022-05-16 11:53:15,2022-03-28 12:40:45,2022-05-16 11:53:15
29,30,NaN,random_batch2_18,random_batch2_18,1,103,2022-05-09 09:33:41,2022-03-28 12:42:10,2022-05-09 09:33:41


Information on when series where completed. 
Perhaps relevant when filtering on annotation step

# MERGING

In [22]:
df = pd.merge(predicted_labels[['filename', 'predictions', 'top_1', 'top_5']], images[['title', 'id']], left_on='filename', right_on='title', how='left')
print(df.shape[0])
df.rename(columns = {'id':'image_id'}, inplace = True)

30498


In [23]:
df.drop(['filename'], axis=1, inplace=True)

In [24]:
df = pd.merge(df, scans[['title','id', 'too_difficult', 'unusable']])
print(df.shape[0])
df.rename(columns = {'id':'scan_id'}, inplace = True)

30498


In [25]:
df = pd.merge(df, indexeer_data[['image_id', 'updated_at', 'gebruiker_id', 'annotated_label']])
print(df.shape[0])

58573


In [26]:
df.rename(columns = {'updated_at':'annotated_on', 'gebruiker_id': 'annotator_id'}, inplace = True)

In [27]:
df = pd.merge(df, controle_data[['image_id', 'updated_at', 'gebruiker_id', 'corrected_label']], on='image_id')

In [28]:
df.rename(columns = {'updated_at':'checked_on', 'gebruiker_id': 'validator_id'}, inplace = True)

In [29]:
df['annotated_label'] = df['annotated_label'].apply(fix_labels)
df['corrected_label'] = df['corrected_label'].apply(fix_labels)

In [30]:
df = pd.merge(df, opmerkingen[['scan_id', 'toelichting', 'gebruiker_id']], left_on='scan_id', right_on='scan_id', how='left')

In [31]:
disagreement = {}
for i, group in df.groupby('image_id'):
    if group['annotated_label'].values[0] != group['annotated_label'].values[1]:
        disagreement[group['image_id'].values[0]] = 'disagree'
    else:
        disagreement[group['image_id'].values[0]] = 'agree'

In [32]:
disagree_df = pd.DataFrame(disagreement, index=['agreement']).T

In [33]:
df = pd.merge(df, disagree_df, left_on='image_id', right_index=True)

In [34]:
in_top_1 = []
in_top_5 = []

for i, row in df.iterrows():
    if row['corrected_label'] == row['top_1']:
        in_top_1.append(1)
        in_top_5.append(1)
    elif row['corrected_label'] in row['top_5']:
        in_top_1.append(0)
        in_top_5.append(1)
    else:
        in_top_1.append(0)
        in_top_5.append(0)
        

In [35]:
df['in_top_1'] = in_top_1
df['in_top_5'] = in_top_5

In [36]:
df[(df['corrected_label'] == 'no_description_found') & (df['toelichting'].notnull())].drop_duplicates(subset=['title']).to_csv('no_desc.csv')

In [38]:
df.to_csv('output_step2.csv', index=False)

### prepare training data

In [201]:
data_path = glob.glob('/Users/melvinwevers/Sync/Work/data/DeBoer/VeleHanden/**/**/*.jpg')
export_path = '../data/step2/'

In [203]:
for _ in data_path:
    filename = os.path.basename(_)[:-4]
    try:
        
        label = df[df['title'] == filename]['corrected_label'].values[0]
        path = os.path.join(export_path, label)
        if os.path.isdir(path):
            shutil.copy(_, path)
        else:
            os.mkdir(path)
            shutil.copy(_, path)
    except:
        pass
        

In [1]:
df

NameError: name 'df' is not defined